In [ ]:
%pip install -U starpoint

In [ ]:
from starpoint.db import Client
from starpoint.openai import OpenAIClient


In [ ]:
import os 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
STARPOINT_API_KEY = os.environ["STARPOINT_API_KEY"]

In [ ]:
starpoint_client = Client(api_key=STARPOINT_API_KEY)

starpoint_openai_client = OpenAIClient(starpoint=starpoint_client, openai_key=OPENAI_API_KEY)


In [ ]:
import json

documents_json = None

with open('./monster_text.json') as f:
    documents_json = json.loads(f)

documents = []

for 